In [2]:
import sys
sys.path.append("../..")

import os
import pickle
import random

from tqdm import tqdm

from time import sleep

In [3]:
import pandas as pd
import requests

def listar_codigos_alimentos_tbca(pagina: str) -> list:
    """
    Extrai os códigos dos alimentos de uma página específica da TBCA.

    Args:
        url_pagina (str): A URL completa da página da TBCA contendo a tabela.
                          Ex: 'http://www.tbca.net.br/base-dados/composicao_alimentos.php?pagina=1'

    Returns:
        list: Uma lista de strings com todos os códigos dos alimentos encontrados.
              Retorna uma lista vazia se a tabela não for encontrada ou em caso de erro.
    """
    
    url = f'http://www.tbca.net.br/base-dados/composicao_alimentos.php?pagina={pagina}'
    
    try:
        # Adicionar um User-Agent para simular um navegador e evitar bloqueios simples.
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        # O pandas lê todas as tabelas HTML da página e retorna uma lista de DataFrames.
        todas_as_tabelas = pd.read_html(requests.get(url, headers=headers).content)
        
        # Procura pela tabela correta, que é a que contém a coluna 'Código'.
        tabela_alimentos = None
        for tabela in todas_as_tabelas:
            if 'Código' in tabela.columns:
                tabela_alimentos = tabela
                break # Encontrou a tabela, pode sair do loop.

        # Se a tabela foi encontrada, extrai os códigos.
        if tabela_alimentos is not None:
            # Converte a coluna 'Código' para uma lista de strings.
            # Usamos .astype(str) para garantir que não haja problemas com formatação (ex: 1234.0)
            codigos = tabela_alimentos['Código'].astype(str).tolist()
            return codigos
        else:
            print(f"Aviso: Nenhuma tabela com a coluna 'Código' foi encontrada em {url}")
            return []

    except Exception as e:
        print(f"Ocorreu um erro ao processar a URL '{url}': {e}")
        return []

In [4]:
def listar_dados_alimentos_tbca(numero_pagina: int) -> list[tuple]:
    """
    Extrai uma tupla (Código, Nome, Grupo) dos alimentos
    de uma página específica da TBCA.

    Args:
        numero_pagina (int): O número da página da TBCA a ser consultada.

    Returns:
        list[tuple]: Uma lista de tuplas, onde cada tupla contém
                     (Código, Nome, Grupo).
                     Retorna uma lista vazia em caso de erro.
    """
    url_pagina = f'http://www.tbca.net.br/base-dados/composicao_alimentos.php?pagina={numero_pagina}'
    
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        todas_as_tabelas = pd.read_html(requests.get(url_pagina, headers=headers).content)
        
        tabela_alimentos = None
        
        # ***** A CORREÇÃO ESTÁ AQUI *****
        # O nome da coluna é 'Nome', e não 'Nome do alimento'.
        colunas_necessarias = ['Código', 'Nome', 'Grupo']

        for tabela in todas_as_tabelas:
            if all(coluna in tabela.columns for coluna in colunas_necessarias):
                tabela_alimentos = tabela
                break

        if tabela_alimentos is not None:
            df_selecionado = tabela_alimentos[colunas_necessarias]
            lista_de_tuplas = list(df_selecionado.itertuples(index=False, name=None))
            return lista_de_tuplas
        else:
            print(f"Aviso: Nenhuma tabela com as colunas {colunas_necessarias} foi encontrada na página {numero_pagina}")
            return []

    except ValueError:
        # Erro comum quando a página não tem tabelas
        print(f"Aviso: Nenhuma tabela foi encontrada pelo Pandas na página {numero_pagina}.")
        return []
    except Exception as e:
        print(f"Ocorreu um erro ao processar a página {numero_pagina}: {e}")
        return []


In [5]:
listar_dados_alimentos_tbca(1)

[('BRC0001C', 'Abacate, polpa, in natura, Brasil', 'Frutas e derivados'),
 ('BRC0195C', 'Abacaxi, cozido, caramelado', 'Frutas e derivados'),
 ('BRC0300C',
  'Abacaxi, grelhado, c/ cobertura de chocolate',
  'Frutas e derivados'),
 ('BRC0054C', 'Abacaxi, polpa, congelada, Brasil', 'Frutas e derivados'),
 ('BRC0264C', 'Abacaxi, polpa, desidratada, Brasil', 'Frutas e derivados'),
 ('BRC0218C', 'Abacaxi, polpa, grelhado, c/ canela', 'Frutas e derivados'),
 ('BRC0002C', 'Abacaxi, polpa, in natura, Brasil', 'Frutas e derivados'),
 ('BRC0069C', 'Abiu, in natura, Brasil', 'Frutas e derivados'),
 ('BRC0680B',
  'Abóbora, cabotian (japonesa), c/ quiabo, refogado (c/ óleo, cebola e alho), c/ salsa, c/ sal',
  'Vegetais e derivados'),
 ('BRC0059B',
  'Abóbora, cabotian (japonesa), s/ casca, cozida, drenada, s/ óleo, s/ sal, Brasil',
  'Vegetais e derivados'),
 ('BRC0002B',
  'Abóbora, cabotian (japonesa), s/ casca, s/sementes, crua, Brasil',
  'Vegetais e derivados'),
 ('BRC0003B',
  'Abóbora, me

In [14]:
def get_codigos_tbca() -> list:
    nome_arquivo_pickle = 'dados_completos_tbca.pkl'

    if os.path.exists(nome_arquivo_pickle):
        print(f"O arquivo '{nome_arquivo_pickle}' já existe.")
        print("O web scraping não será executado novamente.")
        
        with open(nome_arquivo_pickle, 'rb') as f:
            dados_existentes = pickle.load(f)
        print(f"\nVerificação: {len(dados_existentes)} registros já estão salvos.")
        if dados_existentes:
            print("Exemplo do primeiro registro:", dados_existentes[0])

        return dados_existentes
    else:
        print(f"Arquivo '{nome_arquivo_pickle}' não encontrado. Iniciando a coleta de dados da TBCA...")
        
        todos_os_dados = []
        # O site informa um total de 10 páginas na paginação, mas os dados vão além. 
        # Vamos manter 57 por enquanto, que parece ser o total real de páginas com dados.
        total_paginas = 57 

        for i in range(1, total_paginas + 1):
            print(f"Processando página {i}/{total_paginas}...")
            dados_da_pagina = listar_dados_alimentos_tbca(i)
            
            if dados_da_pagina:
                todos_os_dados.extend(dados_da_pagina)
            else:
                # Se uma página não retornar dados, podemos parar para não fazer requisições desnecessárias.
                print(f"Página {i} não retornou dados. Finalizando a busca.")
                break
                
            sleep(0.5)

        print("\nColeta finalizada.")
        print(f"Total de registros de alimentos encontrados: {len(todos_os_dados)}")

        if todos_os_dados:
            try:
                with open(nome_arquivo_pickle, 'wb') as f:
                    pickle.dump(todos_os_dados, f)
                print(f"Lista de dados salva com sucesso no arquivo '{nome_arquivo_pickle}'")
                
                return todos_os_dados
            except Exception as e:
                print(f"Ocorreu um erro ao salvar o arquivo pickle: {e}")
                return []
        else:
            print("Nenhum dado foi coletado. O arquivo pickle não foi criado.")
            return []
        
codigos_carregados = get_codigos_tbca()

O arquivo 'dados_completos_tbca.pkl' já existe.
O web scraping não será executado novamente.

Verificação: 5687 registros já estão salvos.
Exemplo do primeiro registro: ('BRC0001C', 'Abacate, polpa, in natura, Brasil', 'Frutas e derivados')


In [15]:
codigos_carregados

print("\nVerificação: Primeiros 10 códigos carregados do arquivo:")
print(codigos_carregados[:10])
print(len(codigos_carregados))
print(len(set(codigos_carregados)))


Verificação: Primeiros 10 códigos carregados do arquivo:
[('BRC0001C', 'Abacate, polpa, in natura, Brasil', 'Frutas e derivados'), ('BRC0195C', 'Abacaxi, cozido, caramelado', 'Frutas e derivados'), ('BRC0300C', 'Abacaxi, grelhado, c/ cobertura de chocolate', 'Frutas e derivados'), ('BRC0054C', 'Abacaxi, polpa, congelada, Brasil', 'Frutas e derivados'), ('BRC0264C', 'Abacaxi, polpa, desidratada, Brasil', 'Frutas e derivados'), ('BRC0218C', 'Abacaxi, polpa, grelhado, c/ canela', 'Frutas e derivados'), ('BRC0002C', 'Abacaxi, polpa, in natura, Brasil', 'Frutas e derivados'), ('BRC0069C', 'Abiu, in natura, Brasil', 'Frutas e derivados'), ('BRC0680B', 'Abóbora, cabotian (japonesa), c/ quiabo, refogado (c/ óleo, cebola e alho), c/ salsa, c/ sal', 'Vegetais e derivados'), ('BRC0059B', 'Abóbora, cabotian (japonesa), s/ casca, cozida, drenada, s/ óleo, s/ sal, Brasil', 'Vegetais e derivados')]
5687
5687


In [18]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def extrair_detalhes_alimento(alimento_info: tuple) -> dict:
    """
    Extrai os detalhes nutricionais de um alimento e já retorna o dicionário
    em formato "achatado", pronto para o DataFrame.

    Args:
        alimento_info (tuple): Uma tupla contendo (código, nome, grupo) do alimento.

    Returns:
        dict: Um dicionário "achatado" com o código, nome, grupo e cada
              componente nutricional como uma chave de nível superior.
              Retorna None em caso de erro.
    """
    codigo, nome, grupo = alimento_info
    url_detalhes = f'https://www.tbca.net.br/base-dados/int_composicao_alimentos_2_edit.php?cod_produto={codigo}'
    
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        response = requests.get(url_detalhes, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        
        tabela_nutrientes = soup.find('table', id='tabela1')

        if not tabela_nutrientes:
            print(f"Aviso: Tabela de nutrientes não encontrada para o código {codigo}")
            return None

        componentes = {}
        for linha in tabela_nutrientes.find('tbody').find_all('tr'):
            celulas = linha.find_all('td')
            if len(celulas) > 2:
                nome_componente = celulas[0].get_text(strip=True)
                valor_texto = celulas[2].get_text(strip=True)
                if valor_texto and valor_texto.lower() not in ['tr', 'na', '-']:
                    try:
                        valor_float = float(valor_texto.replace(',', '.'))
                        componentes[nome_componente] = valor_float
                    except (ValueError, TypeError):
                        pass
        
        if not componentes:
            print(f"Aviso: Nenhum componente válido encontrado para o código {codigo}")
            return None

        # ***** LÓGICA DE TRANSFORMAÇÃO MOVIDA PARA DENTRO DA FUNÇÃO *****
        # 1. Cria o dicionário base
        dados_finais = {
            'codigo': codigo,
            'descricao': nome,
            'grupo': grupo
        }
        
        # 2. Atualiza o dicionário base com os componentes, achatando a estrutura
        dados_finais.update(componentes)
        
        # 3. Retorna o dicionário já formatado
        return dados_finais

    except Exception as e:
        print(f"Erro ao processar o código {codigo}: {e}")
        return None
    
extrair_detalhes_alimento( ("BRC0001C", "Abacate, polpa, in natura, Brasil", "Frutas e derivados") )

{'codigo': 'BRC0001C',
 'descricao': 'Abacate, polpa, in natura, Brasil',
 'grupo': 'Frutas e derivados',
 'Energia': 76.0,
 'Umidade': 86.3,
 'Carboidrato total': 5.84,
 'Carboidrato disponível': 1.81,
 'Proteína': 1.15,
 'Lipídios': 6.21,
 'Fibra alimentar': 4.03,
 'Álcool': 0.0,
 'Cinzas': 0.47,
 'Colesterol': 0.0,
 'Ácidos graxos saturados': 1.7,
 'Ácidos graxos monoinsaturados': 3.18,
 'Ácidos graxos poliinsaturados': 1.04,
 'Ácidos graxos trans': 0.0,
 'Cálcio': 7.16,
 'Ferro': 0.18,
 'Magnésio': 17.0,
 'Fósforo': 18.5,
 'Potássio': 174.0,
 'Zinco': 0.23,
 'Cobre': 0.12,
 'Selênio': 0.2,
 'Vitamina A (RE)': 6.21,
 'Vitamina A (RAE)': 3.11,
 'Vitamina D': 0.0,
 'Alfa-tocoferol (Vitamina E)': 0.02,
 'Riboflavina': 0.04,
 'Vitamina B12': 0.0,
 'Vitamina C': 7.32,
 'Equivalente de folato': 41.5,
 'Sal de adição': 0.0,
 'Açúcar de adição': 0.0,
 'Gordura de adição': 0.0,
 'Proteína vegetal': 1.15,
 'Proteína animal': 0.0}

In [16]:
def get_dados_tbca() -> pd.DataFrame:
    # Arquivo de entrada (gerado no script anterior)
    arquivo_lista_alimentos = 'dados_completos_tbca.pkl'
    # Arquivo de saída (DataFrame com todos os detalhes)
    arquivo_dataframe_final = 'dataframe_nutricional_tbca.pkl'

    # Verifica se o DataFrame final já existe
    if os.path.exists(arquivo_dataframe_final):
        print(f"O arquivo final '{arquivo_dataframe_final}' já existe.")
        print("Para gerar novamente, apague o arquivo manualmente.")
        
        # Exemplo de como carregar e visualizar o DataFrame
        df_final = pd.read_pickle(arquivo_dataframe_final)
        print("\nDataFrame carregado com sucesso!")
        print(f"Total de alimentos no DataFrame: {len(df_final)}")
        print("Amostra:")
        print(df_final.head())
        return df_final

    else:
        # Verifica se o arquivo de entrada existe
        if not os.path.exists(arquivo_lista_alimentos):
            print(f"Erro: Arquivo de entrada '{arquivo_lista_alimentos}' não encontrado.")
            print("Por favor, execute o script anterior primeiro para gerar a lista de alimentos.")
            return pd.DataFrame()
        else:
            print(f"Carregando a lista de alimentos de '{arquivo_lista_alimentos}'...")
            with open(arquivo_lista_alimentos, 'rb') as f:
                lista_alimentos = pickle.load(f)
            
            print(f"{len(lista_alimentos)} alimentos para processar. Iniciando coleta de detalhes...")
            
            # Lista para armazenar todos os dicionários de alimentos detalhados
            dados_detalhados_lista = []
            
            # Itera sobre a lista de alimentos com uma barra de progresso (tqdm)
            for alimento in tqdm(lista_alimentos, desc="Coletando detalhes"):
                detalhes = extrair_detalhes_alimento(alimento)
                if detalhes:
                    dados_detalhados_lista.append(detalhes)
                sleep(0.2) # Pausa para não sobrecarregar o servidor
            
            print("\nColeta de detalhes finalizada.")

            if dados_detalhados_lista:
                # Cria o DataFrame do pandas a partir da lista de dicionários
                df_final = pd.DataFrame(dados_detalhados_lista)
                
                print("DataFrame criado com sucesso!")
                print(f"Total de registros processados: {len(df_final)}")
                
                # Salvando o DataFrame como um arquivo pickle
                try:
                    df_final.to_pickle(arquivo_dataframe_final)
                    print(f"DataFrame salvo com sucesso no arquivo '{arquivo_dataframe_final}'")
                    print("\nAmostra do DataFrame final:")
                    print(df_final.head())
                except Exception as e:
                    print(f"Ocorreu um erro ao salvar o DataFrame: {e}")
                    
                return df_final
            else:
                print("Nenhum dado detalhado foi coletado. O DataFrame não foi criado.")
                return pd.DataFrame()


df_final = get_dados_tbca()

O arquivo final 'dataframe_nutricional_tbca.pkl' já existe.
Para gerar novamente, apague o arquivo manualmente.

DataFrame carregado com sucesso!
Total de alimentos no DataFrame: 5658
Amostra:
     codigo                                     descricao               grupo   
0  BRC0001C             Abacate, polpa, in natura, Brasil  Frutas e derivados  \
1  BRC0195C                   Abacaxi, cozido, caramelado  Frutas e derivados   
2  BRC0300C  Abacaxi, grelhado, c/ cobertura de chocolate  Frutas e derivados   
3  BRC0054C             Abacaxi, polpa, congelada, Brasil  Frutas e derivados   
4  BRC0264C           Abacaxi, polpa, desidratada, Brasil  Frutas e derivados   

   Energia  Umidade  Carboidrato total  Carboidrato disponível  Proteína   
0     76.0     86.3               5.84                    1.81      1.15  \
1    136.0     65.5              33.30                   32.50      0.60   
2    148.0     69.9              21.30                   20.20      1.69   
3     33.0     9

In [17]:
print(df_final.head(5))

     codigo                                     descricao               grupo   
0  BRC0001C             Abacate, polpa, in natura, Brasil  Frutas e derivados  \
1  BRC0195C                   Abacaxi, cozido, caramelado  Frutas e derivados   
2  BRC0300C  Abacaxi, grelhado, c/ cobertura de chocolate  Frutas e derivados   
3  BRC0054C             Abacaxi, polpa, congelada, Brasil  Frutas e derivados   
4  BRC0264C           Abacaxi, polpa, desidratada, Brasil  Frutas e derivados   

   Energia  Umidade  Carboidrato total  Carboidrato disponível  Proteína   
0     76.0     86.3               5.84                    1.81      1.15  \
1    136.0     65.5              33.30                   32.50      0.60   
2    148.0     69.9              21.30                   20.20      1.69   
3     33.0     91.3               7.80                    7.47      0.47   
4    279.0     27.5              64.90                   58.60      3.80   

   Lipídios  Fibra alimentar  ...  Equivalente de folato

{'codigo': 'BRC0001C',
 'descricao': 'Abacate, polpa, in natura, Brasil',
 'grupo': 'Frutas e derivados',
 'Energia': 76.0,
 'Umidade': 86.3,
 'Carboidrato total': 5.84,
 'Carboidrato disponível': 1.81,
 'Proteína': 1.15,
 'Lipídios': 6.21,
 'Fibra alimentar': 4.03,
 'Álcool': 0.0,
 'Cinzas': 0.47,
 'Colesterol': 0.0,
 'Ácidos graxos saturados': 1.7,
 'Ácidos graxos monoinsaturados': 3.18,
 'Ácidos graxos poliinsaturados': 1.04,
 'Ácidos graxos trans': 0.0,
 'Cálcio': 7.16,
 'Ferro': 0.18,
 'Magnésio': 17.0,
 'Fósforo': 18.5,
 'Potássio': 174.0,
 'Zinco': 0.23,
 'Cobre': 0.12,
 'Selênio': 0.2,
 'Vitamina A (RE)': 6.21,
 'Vitamina A (RAE)': 3.11,
 'Vitamina D': 0.0,
 'Alfa-tocoferol (Vitamina E)': 0.02,
 'Riboflavina': 0.04,
 'Vitamina B12': 0.0,
 'Vitamina C': 7.32,
 'Equivalente de folato': 41.5,
 'Sal de adição': 0.0,
 'Açúcar de adição': 0.0,
 'Gordura de adição': 0.0,
 'Proteína vegetal': 1.15,
 'Proteína animal': 0.0}

In [13]:
 refeicao = "Maçã"
 mask = df_final['descricao'].str.contains(refeicao, case=False, na=False)
 df_final[mask]

,codigo,descricao,grupo,Energia,Umidade,Carboidrato total,Carboidrato disponível,Proteína,Lipídios,Fibra alimentar,...,Equivalente de folato,Sal de adição,Açúcar de adição,Gordura de adição,Proteína vegetal,Proteína animal,Sódio,Tiamina,Niacina,Vitamina B6
313,BRC0278C,"Banana, cozida, c/ manteiga, c/ açúcar (média ...",Frutas e derivados,156.0,62.5,32.40,30.2,1.41,2.81,2.29,...,14.40,4.5,0.0,NaN,NaN,NaN,0.73,0.01,0.01,0.13
315,BRC0277C,"Banana, cozida, s/ manteiga, c/ açúcar (média ...",Frutas e derivados,119.0,64.2,33.30,31.0,1.44,0.24,2.36,...,14.80,4.5,0.0,NaN,NaN,NaN,0.61,0.01,0.01,0.14
316,BRC0276C,"Banana, cozida, s/ manteiga, s/ açúcar (média ...",Frutas e derivados,124.0,67.4,30.00,27.8,1.48,0.24,2.20,...,15.60,0.0,0.0,NaN,NaN,NaN,0.00,0.01,0.00,0.14
323,BRC0280C,"Banana, grelhada, c/ manteiga, c/ açúcar e can...",Frutas e derivados,161.0,61.2,33.70,31.3,1.47,2.82,2.39,...,15.10,4.5,0.0,NaN,NaN,NaN,0.73,0.01,0.01,0.13
324,BRC0281C,"Banana, grelhada, s/ manteiga, c/ açúcar e can...",Frutas e derivados,142.0,62.7,34.60,32.2,1.50,0.25,2.45,...,15.50,4.5,0.0,NaN,NaN,NaN,0.61,0.01,0.01,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,BRC0106G,"Vitamina, leite de vaca integral c/ banana, ma...",Leite e derivados,108.0,74.3,20.50,19.3,2.54,2.02,1.23,...,5.68,0.0,6.0,NaN,NaN,NaN,30.40,0.06,0.96,0.02
5630,BRC0105G,"Vitamina, leite de vaca integral c/ banana, ma...",Leite e derivados,83.0,80.7,13.80,12.5,2.67,2.15,1.29,...,5.88,0.0,0.0,NaN,NaN,NaN,32.00,0.06,1.02,0.02
5631,BRC0082G,"Vitamina, leite de vaca integral c/ banana, ma...",Leite e derivados,64.0,85.3,10.60,9.8,1.85,1.73,0.75,...,2.70,0.0,0.0,NaN,NaN,NaN,33.70,0.03,0.79,0.02
5632,BRC0085G,"Vitamina, leite de vaca integral c/ maçã, c/ a...",Leite e derivados,79.0,82.0,13.30,12.9,2.00,2.10,0.40,...,3.37,0.0,5.0,NaN,NaN,NaN,42.10,0.03,0.99,0.01


In [22]:
df_final.columns

Index(['codigo', 'descricao', 'grupo', 'Energia', 'Umidade',
       'Carboidrato total', 'Carboidrato disponível', 'Proteína', 'Lipídios',
       'Fibra alimentar', 'Álcool', 'Cinzas', 'Colesterol',
       'Ácidos graxos saturados', 'Ácidos graxos monoinsaturados',
       'Ácidos graxos poliinsaturados', 'Ácidos graxos trans', 'Cálcio',
       'Ferro', 'Magnésio', 'Fósforo', 'Potássio', 'Zinco', 'Cobre', 'Selênio',
       'Vitamina A (RE)', 'Vitamina A (RAE)', 'Vitamina D',
       'Alfa-tocoferol (Vitamina E)', 'Riboflavina', 'Vitamina B12',
       'Vitamina C', 'Equivalente de folato', 'Sal de adição',
       'Açúcar de adição', 'Gordura de adição', 'Proteína vegetal',
       'Proteína animal', 'Sódio', 'Tiamina', 'Niacina', 'Vitamina B6'],
      dtype='object')